### CV mediante video

In [1]:
import cv2
import mediapipe as mp
import time

In [3]:
cap = cv2.VideoCapture(0)
mp_drawing = mp.solutions.drawing_utils
mp_heads = mp.solutions.hands
ftime = 0
with mp_heads.Hands(static_image_mode = True,
                    max_num_hands = 2,
                    min_detection_confidence = 0.5) as hands:
    while True :
        _, frame = cap.read()
        h, w, _ = frame.shape
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(image_rgb)
        if results.multi_hand_landmarks is not None :
            for hands_landmarks in results.multi_hand_landmarks :
                # Mostrar los puntos
                # print(hands_landmarks) 
                # DIBUJAR SOLO ALGUNOS PUNTOS
                # Convertir a coordenas de pixeles
                x1 = int(hands_landmarks.landmark[mp_heads.HandLandmark.THUMB_TIP].x*w)
                y1 = int(hands_landmarks.landmark[mp_heads.HandLandmark.THUMB_TIP].y*h)

                x2 = int(hands_landmarks.landmark[mp_heads.HandLandmark.INDEX_FINGER_TIP].x*w)
                y2 = int(hands_landmarks.landmark[mp_heads.HandLandmark.INDEX_FINGER_TIP].y*h)

                x3 = int(hands_landmarks.landmark[mp_heads.HandLandmark.MIDDLE_FINGER_TIP].x*w)
                y3 = int(hands_landmarks.landmark[mp_heads.HandLandmark.MIDDLE_FINGER_TIP].y*h)

                x4 = int(hands_landmarks.landmark[mp_heads.HandLandmark.RING_FINGER_TIP].x*w)
                y4 = int(hands_landmarks.landmark[mp_heads.HandLandmark.RING_FINGER_TIP].y*h)

                x5 = int(hands_landmarks.landmark[mp_heads.HandLandmark.PINKY_TIP].x*w)
                y5 = int(hands_landmarks.landmark[mp_heads.HandLandmark.PINKY_TIP].y*h)

                cv2.circle(frame, (x1, y1), 5, color=(0, 0, 255), thickness=5)
                cv2.circle(frame, (x2, y2), 5, color=(0, 255, 255), thickness=5)
                cv2.circle(frame, (x3, y3), 5, color=(255, 0, 255), thickness=5)
                cv2.circle(frame, (x4, y4), 5, color=(255, 0, 0), thickness=5)
                cv2.circle(frame, (x5, y5), 5, color=(255, 255, 0), thickness=5)

                # DIBUJAR TODOS LOS PUNTOS
                # mp_drawing.draw_landmarks(frame, hands_landmarks, 
                #                       mp_heads.HAND_CONNECTIONS, 
                #                       # Modificar el tamaño de los circulos
                #                       mp_drawing.DrawingSpec(color=(125, 120, 10), thickness=3, circle_radius=5), 
                #                       # Modificar los conectores
                #                       mp_drawing.DrawingSpec(color=(20, 45, 10), thickness=3))

        # Imprime la resolución de la cámara web
        # print(frame.shape)
        ctime = time.time()
        fps = 1/(ctime-ftime)
        ftime = ctime

        # Permite escribir un texto en la imagen mostrada
        """
            1P: Video
            2P: Texto a mostrar
            3P: Ubicacion: (AnchoxAlto)
            4P: Estilo de fuente
            5P: Tamaño de la letra (1.5)
            6P: El color de la fuente en BGR
        """
        cv2.putText(frame, f'fps:{int(fps)}', (500, 30), cv2.FONT_HERSHEY_PLAIN, 1.5, (255, 0, 0), 2)

        
        # Mostrar el video
        cv2.imshow("Captura de las manos", frame)
        if cv2.waitKey(1) & 0XFF==ord("q") :
            break

cv2.destroyAllWindows()
# cap.release()

### Contar la cantidad de dedos levantados

In [11]:
cap = cv2.VideoCapture(0)
mp_drawing = mp.solutions.drawing_utils
mp_heads = mp.solutions.hands
# Arreglo del número de punto para la mano según la librería
puntas = [4, 8, 12, 16, 20]
ftime = 0

with mp_heads.Hands(static_image_mode = True,
                    max_num_hands = 2,
                    min_detection_confidence = 0.5) as hands:
    while True :
        _, frame = cap.read()
        h, w, _ = frame.shape
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(image_rgb)
        lista = []
        if results.multi_hand_landmarks is not None :
            # toma el primer resultado [0] (primera mano en detectar)
            myhand = results.multi_hand_landmarks[0]
            # Iterando las conexiones y puntos de la mano escogida
            for id, lm in enumerate(myhand.landmark) : 
                # Esta es otra forma de definir la ubicación de los puntos a dibujar, luego de detectar la mano
                x, y = int(lm.x*w), int(lm.y*h)
                lista.append([id, x, y])

            mp_drawing.draw_landmarks(frame, myhand,
                                      mp_heads.HAND_CONNECTIONS)

        fingers = []
        # Si lista no esta vacia, es que se detecto una mano
        if len(lista) != 0 :
            # Establece la condicion para determinar si la punto del dedo esta doblada (si esta antes del 3)
            # 1 -> x
            if lista[puntas[0]][1]  > lista[puntas[0] -1][1] :
                fingers.append(1)
            else :
                fingers.append(0)
            for id in range(1, 5) :
            # 2 -> y
                if lista[puntas[id]][2] < lista[puntas[id]-2][2] :
                    fingers.append(1)
                else :
                    fingers.append(0)

            total = fingers.count(1)

            mensaje = ""
            if total == 0 :
                cv2.rectangle(frame, (5, 5), (445, 35), (0, 255, 0), cv2.FILLED)
                mensaje = "0. Motor Listo - Estatico"
            if total == 1 :
                cv2.rectangle(frame, (5, 5), (255, 35), (0, 255, 255), cv2.FILLED)
                mensaje = "1. Avanzar"
            if total == 2 :
                cv2.rectangle(frame, (5, 5), (285, 35), (255, 0, 255), cv2.FILLED)
                mensaje = "2. Retroceder "
            if total == 3 :
                cv2.rectangle(frame, (5, 5), (255, 35), (0, 0, 255), cv2.FILLED)
                mensaje = "3. Derecha"
            if total == 4 :
                cv2.rectangle(frame, (5, 5), (265, 35), (45, 90, 105), cv2.FILLED)
                mensaje = "4. Izquierda"
            if total == 5 :
                cv2.rectangle(frame, (5, 5), (325, 35), (255, 255, 0), cv2.FILLED)
                mensaje = "5. Detener motor"
            
            # print(mensaje)
            cv2.putText(frame, f'Accion: {mensaje}', (10, 30), cv2.FONT_HERSHEY_PLAIN, 1.5, (255, 0, 0), 2)
        ctime = time.time()
        fps = 1/(ctime-ftime)
        ftime = ctime

        # Permite escribir un texto en la imagen mostrada
        """
            1P: Video
            2P: Texto a mostrar
            3P: Ubicacion: (AnchoxAlto)
            4P: Estilo de fuente
            5P: Tamaño de la letra (1.5)
            6P: El color de la fuente en BGR
        """
        cv2.putText(frame, f'fps:{int(fps)}', (500, 30), cv2.FONT_HERSHEY_PLAIN, 1.5, (255, 0, 0), 2)

        
        # Mostrar el video
        cv2.imshow("Captura de las manos", frame)
        if cv2.waitKey(1) & 0XFF==ord("q") :
            break

cv2.destroyAllWindows()
# Apagar cámara
cap.release()